## <a id='toc1_1_'></a>[1️⃣ Data Preparation](#toc0_)
<!-- **Designed by:** [datamover.ai](https://www.datamover.ai)  -->

In [1]:
import os

# import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# make script reproducible
np.random.seed(42)

**1. Fetch dataset** 

Download dataset from this [url](https://www.kaggle.com/datasets/thomasnibb/amsterdam-house-price-prediction) and load the data in a `pd.DataFrame`. 

💁‍♂️ Checked this [article](https://www.datamover.ai/post/the-right-way-to-set-absolute-path-in-python) to learn how to load dataset OS agnostic.

In [2]:
DATA_DIR = "data"
FILENAME = "HousingPrices-Amsterdam-August-2021.csv"

data = pd.read_csv(
    os.path.join(DATA_DIR, FILENAME),
    index_col=0,
)
data.head()

,Address,Zip,Price,Area,Room,Lon,Lat
1,"Blasiusstraat 8 2, Amsterdam",1091 CR,685000.0,64,3,4.907736,52.356157
2,"Kromme Leimuidenstraat 13 H, Amsterdam",1059 EL,475000.0,60,3,4.850476,52.348586
3,"Zaaiersweg 11 A, Amsterdam",1097 SM,850000.0,109,4,4.944774,52.343782
4,"Tenerifestraat 40, Amsterdam",1060 TH,580000.0,128,6,4.789928,52.343712
5,"Winterjanpad 21, Amsterdam",1036 KN,720000.0,138,5,4.902503,52.410538


**2. Check dataset size and ensure workspace has enough storage even when dealing with big datasets**

In [3]:
size_b = data.memory_usage(deep=True).sum()  # get size in byte
size_mb = size_b / (1024 * 1024)  # convert byte to mb
print(f"Size data: {size_mb:.2f} Mb")

Size data: 0.18 Mb


📝 `deep=True` introspect the data deeply by interrogating object dtypes for system-level memory consumption, and include it in the returned values.

**3. Check type of data (time series, sample, geographical, etc.) and make sure they are what they should be.**

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 924 entries, 1 to 924
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Address  924 non-null    object 
 1   Zip      924 non-null    object 
 2   Price    920 non-null    float64
 3   Area     924 non-null    int64  
 4   Room     924 non-null    int64  
 5   Lon      924 non-null    float64
 6   Lat      924 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 57.8+ KB


**4. If necessary, convert the data to a format that is easy to manipulate (without changing the data itself, e.g. .csv, .json).**

In this case the dataset is already in a format easy to manipulate, i.e., `.csv`

**5. For training of ML models, sample a hold-out set, put it aside, and <font color='#E7240E'>never look at it</font> ⚠️.**

- typical train/test splits are `60/40`, `70/30`, `80/20`;
- it is convenient to store train and test data separately;
- 📝 often test set and hold-out are terms used interchangeably.

<ins> For this project aim to have a 60/40 train/test split ratio. <ins>

In [5]:
TARGET = "Price"  # get target name

# split data in train and test
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=[TARGET]), data[TARGET], test_size=0.40, random_state=42
)

# re-merge X,y for both train and test
data_train = pd.merge(left=y_train, right=X_train, left_index=True, right_index=True)
data_test = pd.merge(left=y_test, right=X_test, left_index=True, right_index=True)

In [6]:
# Check sample size of each set with corresponding percentage
print(
    f"# sample train set: {data_train.shape[0]} ({data_train.shape[0]/len(data):.2f}%) "
)
print(f"# sample test set: {data_test.shape[0]} ({data_test.shape[0]/len(data):.2f}%) ")

# sample train set: 554 (0.60%) 
# sample test set: 370 (0.40%) 


**6. In a data-rich situation, one can create additional splits called *validation set* (usually equal in size to the hold-out set) to estimate the prediction error for model selection. If dealing with a classification problem, one should also put aside two additional sets: *calibration* and *calibration-validation*.**

In [7]:
# Further split the test data in validtion (50%) and test (50%)
X_val, X_test, y_val, y_test = train_test_split(
    data_test.drop(columns=[TARGET]), data_test[TARGET], test_size=0.50, random_state=42
)

# re-merge X,y for both val and test
data_val = pd.merge(left=y_val, right=X_val, left_index=True, right_index=True)
data_test = pd.merge(left=y_test, right=X_test, left_index=True, right_index=True)

In [8]:
# Double check sample size of each set with corresponding percentage
print(
    f"# sample train set: {data_train.shape[0]} ({data_train.shape[0]/len(data):.2f}%) "
)
print(f"# sample val set: {data_val.shape[0]} ({data_val.shape[0]/len(data):.2f}%) ")
print(f"# sample test set: {data_test.shape[0]} ({data_test.shape[0]/len(data):.2f}%) ")

# assert the the size of each set is correct
assert len(data) == len(data_train) + len(data_val) + len(data_test)

# sample train set: 554 (0.60%) 
# sample val set: 185 (0.20%) 
# sample test set: 185 (0.20%) 


**7. ⬇️ Store train and test locally**
- Store both datasets in `data` folder in `csv` format;
- Save train and test set as `data_train.csv` and `data_set.csv`, respectively (and any additional set created in the previous step).
- For each dataset, retain the column names and discard the index if it is not informative.

💁‍♂️ Automate scripts as much as possible for future data analysis.

In [9]:
# create data folder if not exists
if not os.path.exists("data"):
    os.makedirs("data")

# save data
data_train.to_csv(
    path_or_buf="./data/data_train.csv",
    header=True,  # Write out the column names
    index=False,  # discard index as it is not informative
)

data_test.to_csv(
    path_or_buf="./data/data_test.csv",
    header=True,  # Write out the column names
    index=False,  # discard index as it is not informative
)

data_val.to_csv(
    path_or_buf="./data/data_val.csv",
    header=True,  # Write out the column names
    index=False,  # discard index as it is not informative
)